In [1]:
# install libraries if not installed and set environment variables to get access
# to the openai API, restart is needed after first install

import os
try:
  import interpreter
except:
  !pip install open-interpreter

import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")




Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 55.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.8 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━

In [2]:
_set_env('OPENAI_API_KEY')

In [3]:
from interpreter import interpreter
from interpreter import OpenInterpreter



def initiate_agent(system_message, auto_run = False, loop=True):
  '''
  initiate single agent and set base parameters.
  '''
  agent = OpenInterpreter()
  agent.import_computer_api = True
  agent.verbose = True
  agent.conversation_history = []
  agent.system_message = system_message
  agent.os = True
  agent.llm.supports_vision = True
  agent.llm.model = "gpt-4-vision-preview"
  agent.llm.supports_functions = True
  agent.llm.context_window = 110000
  agent.llm.max_tokens = 4096
  agent.auto_run = auto_run
  agent.loop =  loop
  return agent

agent = initiate_agent("""you are an assistant. please provide help to the user""", True, True)
output = agent.chat("what should i visit when in paris")


Output()

In [13]:
# another example but with human decision point
prompt = """
use the requests library to browse the web and find a good stock to invest,
this is the alpha vantage api key: CGQ1ZD9Z8JNU4TMY
start with reading the news but first explore the news response to decide on a next action
"""
news = interpreter.chat(prompt)


Open Interpreter will require approval before running code.

Use interpreter -y to bypass this.

Press CTRL-C to exit.

Output()

Output()

  Would you like to run this code? (y/n)

  yes



In [14]:
query = """write, execute and debug code to say hi"""
interpreter.chat(query)


Open Interpreter will require approval before running code.

Use interpreter -y to bypass this.

Press CTRL-C to exit.

Output()

Output()

  Would you like to run this code? (y/n)

  y


Output()

Output()

[{'role': 'assistant',
  'type': 'message',
  'content': 'To begin, I\'ll write and execute a simple piece of code to output "Hi". Let\'s start with this.\n\nPlan:\n1. Write code to print "Hi".\n2. Execute the code.'},
 {'role': 'assistant',
  'type': 'code',
  'format': 'python',
  'content': "print('Hi')"},
 {'role': 'computer',
  'type': 'console',
  'format': 'output',
  'content': 'Hi\n'},
 {'role': 'assistant',
  'type': 'message',
  'content': 'The code executed successfully and output "Hi". If you have any further tasks or modifications, please let me know how I can assist you next!'}]

In [16]:

# we make our first multi agent system with open interpreter


def initiate_agent(system_message, auto_run = False, loop=True):
  agent = OpenInterpreter()
  agent.import_computer_api = True
  agent.verbose = True
  agent.conversation_history = []
  agent.system_message = system_message
  agent.os = True
  agent.llm.supports_vision = True
  agent.llm.model = "gpt-4-vision-preview"
  agent.llm.supports_functions = True
  agent.llm.context_window = 110000
  agent.llm.max_tokens = 4096
  agent.auto_run = auto_run
  agent.loop =  loop
  return agent

agent_1 = initiate_agent("""you are a programmer and get messages from a reflector.
                          use 'programmer': to start your messages,
                          you have access to the computer interface and can browse the web""", True, True)
agent_2 = initiate_agent( """You need to reflect upon the previous conversation.
                             Use 'reflector:' to start your messages.
                             If you are happy with the answer end with +FINAL+ but give your opinion anyway.
                             """, True, True)

def swap_roles(messages):
    for message in messages:
        if message['role'] == 'user':
            message['role'] = 'assistant'
        elif message['role'] == 'assistant':
            message['role'] = 'user'
    return messages

agents = [agent_1, agent_2]

# Kick off the conversation
messages = [{"role": "user", "type": "message",
             "content": "you can browse the web. i want to have a table about the following https://blockchain.info/ticker. browse the web"}]

# for i in range(2):
while True:
    print(messages)
    if "+FINAL+" in messages[-1]['content']:
        print("computations are done")
        break
    else:
        for agent in agents:
          messages = agent.chat(messages)
          messages = swap_roles(messages)


[{'role': 'user', 'type': 'message', 'content': 'you can browse the web. i want to have a table about the following https://blockchain.info/ticker. browse the web'}]


Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

[{'role': 'user', 'type': 'message', 'content': "reflector: The use of the Python code to fetch the current price of Bitcoin from an online API was successful, and I've provided you with the most recent price in USD. If there's anything more specific you need regarding Bitcoin prices or any other requests, feel free to let me know. \n\n+FINAL+\n\nLet me know what you'd like to do next."}]
computations are done
